# <center> <b> <h1> T-Test </h1></b></center>

In [59]:
from scipy import stats
import pandas as pd 
import numpy as np

from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr


PATH_DATASETS = "../../2_Data_preparation/1_Compute_raw_datasets/output/"
PATH_OUTPUT =  "output/"

In [60]:
def readCsvs():
    datasets = []
    constant = '_Classified'
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        df = pd.read_csv(PATH_DATASETS + feature + constant+ '.csv', index_col=0)
        datasets.append({'name': feature, 'data': df})
    return datasets

# 1. Prepare the data

In [61]:
Datasets = readCsvs() #Area,MeanCurv,Thickness,Volume

In [62]:
print(Datasets[0]['data'].shape)
Datasets[1]['data'].head()

(1108, 149)


,lh_G_and_S_frontomargin_meancurv,lh_G_and_S_occipital_inf_meancurv,lh_G_and_S_paracentral_meancurv,lh_G_and_S_subcentral_meancurv,lh_G_and_S_transv_frontopol_meancurv,lh_G_and_S_cingul-Ant_meancurv,lh_G_and_S_cingul-Mid-Ant_meancurv,lh_G_and_S_cingul-Mid-Post_meancurv,lh_G_cingul-Post-dorsal_meancurv,lh_G_cingul-Post-ventral_meancurv,...,rh_S_pericallosal_meancurv,rh_S_postcentral_meancurv,rh_S_precentral-inf-part_meancurv,rh_S_precentral-sup-part_meancurv,rh_S_suborbital_meancurv,rh_S_subparietal_meancurv,rh_S_temporal_inf_meancurv,rh_S_temporal_sup_meancurv,rh_S_temporal_transverse_meancurv,class
Subject,,,,,,,,,,,,,,,,,,,,,
100206,0.177,0.165,0.144,0.178,0.198,0.160,0.170,0.171,0.219,0.217,...,0.175,0.128,0.145,0.137,0.152,0.159,0.136,0.136,0.156,1.0
100307,0.173,0.150,0.132,0.183,0.218,0.175,0.157,0.160,0.243,0.198,...,0.188,0.152,0.123,0.125,0.157,0.152,0.144,0.137,0.147,-1.0
100408,0.167,0.173,0.137,0.166,0.180,0.169,0.169,0.179,0.241,0.178,...,0.173,0.132,0.142,0.116,0.139,0.177,0.131,0.131,0.137,1.0
100610,0.138,0.153,0.116,0.139,0.157,0.133,0.151,0.154,0.196,0.163,...,0.161,0.125,0.115,0.119,0.139,0.131,0.114,0.122,0.145,1.0
101006,0.152,0.155,0.121,0.153,0.179,0.157,0.138,0.131,0.202,0.175,...,0.122,0.109,0.118,0.112,0.166,0.160,0.135,0.126,0.135,1.0


In [32]:
class_1 = 0
class_2 = 0
for dataset in Datasets:
    name = dataset['name']
    data = dataset['data']
    
    # copy all the subjects that belongs to the class 1 and -1 respectively
    # remember that class is a colum  in the end of the raw dataset classified
    class_1 = data[data['class'] == 1].copy()  
    class_2 = data[data['class'] == -1].copy()
    
    # the [:-1] is ommit the last character, in this case ommit the last colum that in this case is the column Class
    # you can print this and see the last part of each print to compare. 
    #print(df.columns.values)
    #print('\n')
    #print(df_stat.columns.values)
    
    df_stat_ks = pd.DataFrame(columns=data.columns.values[:-1])
    df_stat_mw = pd.DataFrame(columns=data.columns.values[:-1])
    df_stat = pd.DataFrame(columns=data.columns.values[:-1])
    df_stat_validation_2 = pd.DataFrame(columns=data.columns.values[:-1])

    df_stat_ks.at['KS STAT', :] = 0
    df_stat_ks.at['p-value', :] = 0
    
    df_stat_mw.at['MW STAT', :] = 0
    df_stat_mw.at['p-value', :] = 0
    
    df_stat.at['t(Xi)', :] = 0
    df_stat.at['p-value', :] = 0
    
    df_stat_validation_2.at['t(Xi)', :] = 0
    df_stat_validation_2.at['p-value', :] = 0

    #df_stat.head()
    
    # the two-sample Kolmogorov-Smirnov test is one og the most useful and general nonparametric
    # methods for comparing two samples, as it is sensitive to differences un both location
    # and  shape of the empirical cumulative distribution functions  of the two samples
    for column in data.columns.values[:-1]:
        df_stat_ks.at['KS STAT', column], df_stat_ks.at['p-value', column] = stats.ks_2samp(class_1[column], class_2[column])
        df_stat_mw.at['MW STAT', column], df_stat_mw.at['p-value', column] = stats.mannwhitneyu(class_1[column], class_2[column])
        df_stat.at['t(Xi)', column], df_stat_validation.at['p-value', column] = ttest_ind(class_1[column], class_2[column], equal_var=True)
        
        #t third method 
        # Compute the descriptive statistics of a and b.
        abar = class_1[column].mean()
        avar = class_1[column].var(ddof=1)
        na = class_1[column].size
        adof = na - 1

        bbar = class_2[column].mean()
        bvar = class_2[column].var(ddof=1)
        nb = class_2[column].size
        bdof = nb - 1
        
        df_stat_validation_2.at['t(Xi)', column], df_stat_validation_2.at['p-value', column] = ( 
            ttest_ind_from_stats(abar, np.sqrt(avar), na,
                              bbar, np.sqrt(bvar), nb,
                              equal_var=True))
        
    
    # T = Transpose index and columns 
    df_stat_ks = df_stat_ks.T.sort_values(by=['p-value'], ascending = True)
    df_stat = df_stat.T.sort_values(by=['p-value'], ascending = True)
    df_stat_mw = df_stat_mw.T.sort_values(by=['p-value'], ascending = True)
    df_stat_validation_2 = df_stat_validation_2.T.sort_values(by=['p-value'], ascending = True)
    
    #df_stat.head()
    
    # returns the las n rows, by defect n = 5 
    #df_stat.tail()
    
    #saving the results 
    df_stat.to_csv(PATH_OUTPUT + "T-test_" + name + ".csv")
    df_stat_mw.to_csv(PATH_OUTPUT + "T-test_validation" + name + ".csv")
    df_stat_validation_2.to_csv(PATH_OUTPUT + "T-test_validation_2" + name + ".csv")    

In [94]:
class_1 = 0
class_2 = 0
targets = ['norm', 'expon', 'logistic', 'gumbel', 'gumbel_l', 'gumbel_r', 'extreme1']
result_targets = []
for dataset in Datasets:
    name = dataset['name']
    data = dataset['data']
    
    # copy all the subjects that belongs to the class 1 and -1 respectively
    # remember that class is a colum  in the end of the raw dataset classified
    class_1 = data[data['class'] == 1].copy()  
    class_2 = data[data['class'] == -1].copy()
    
    # the [:-1] is ommit the last character, in this case ommit the last colum that in this case is the column Class
    # you can print this and see the last part of each print to compare. 
    #print(df.columns.values)
    #print('\n')
    #print(df_stat.columns.values)
    
    df_anderson = pd.DataFrame(columns=('region', 'anderson stat', 'critical values', 'significance level'))
    df_anderson['region'] = data.columns.values[:-1]
    df_anderson.set_index('region', inplace=True)
    
    for target in targets:
        for column in data.columns.values[:-1]:
            df_anderson.at[column, 'anderson stat'], df_anderson.at[column, 'critical values'],  df_anderson.at[column, 'significance level'] = stats.anderson(class_1[column], dist = target)

        #df_anderson = df_anderson.T
        result_targets.append({'name': target, 'data': df_anderson})

/home/students/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [113]:
for result in result_targets:
    print(result.head(10))

                                 anderson stat  \
region                                           
lh_G_and_S_frontomargin_area            7.7051   
lh_G_and_S_occipital_inf_area          9.26205   
lh_G_and_S_paracentral_area            15.8826   
lh_G_and_S_subcentral_area             12.7282   
lh_G_and_S_transv_frontopol_area       12.2633   
lh_G_and_S_cingul-Ant_area             7.98115   
lh_G_and_S_cingul-Mid-Ant_area         19.1091   
lh_G_and_S_cingul-Mid-Post_area        18.4801   
lh_G_cingul-Post-dorsal_area           16.5142   
lh_G_cingul-Post-ventral_area          69.8297   

                                                    critical values  \
region                                                                
lh_G_and_S_frontomargin_area      [0.47, 0.632, 0.751, 0.87, 1.029]   
lh_G_and_S_occipital_inf_area     [0.47, 0.632, 0.751, 0.87, 1.029]   
lh_G_and_S_paracentral_area       [0.47, 0.632, 0.751, 0.87, 1.029]   
lh_G_and_S_subcentral_area        [0.47, 0.6

In [114]:
result_targets[0].head(10)

,anderson stat,critical values,significance level
region,,,
lh_G_and_S_frontomargin_area,7.7051,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_occipital_inf_area,9.26205,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_paracentral_area,15.8826,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_subcentral_area,12.7282,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_transv_frontopol_area,12.2633,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_cingul-Ant_area,7.98115,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_cingul-Mid-Ant_area,19.1091,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_and_S_cingul-Mid-Post_area,18.4801,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"
lh_G_cingul-Post-dorsal_area,16.5142,"[0.47, 0.632, 0.751, 0.87, 1.029]","[25.0, 10.0, 5.0, 2.5, 1.0]"


In [68]:
for target in result_targets:
    print(target.head(10))

                             region anderson stat critical values  \
0      lh_G_and_S_frontomargin_area           NaN             NaN   
1     lh_G_and_S_occipital_inf_area           NaN             NaN   
2       lh_G_and_S_paracentral_area           NaN             NaN   
3        lh_G_and_S_subcentral_area           NaN             NaN   
4  lh_G_and_S_transv_frontopol_area           NaN             NaN   
5        lh_G_and_S_cingul-Ant_area           NaN             NaN   
6    lh_G_and_S_cingul-Mid-Ant_area           NaN             NaN   
7   lh_G_and_S_cingul-Mid-Post_area           NaN             NaN   
8      lh_G_cingul-Post-dorsal_area           NaN             NaN   
9     lh_G_cingul-Post-ventral_area           NaN             NaN   

  significance level  
0                NaN  
1                NaN  
2                NaN  
3                NaN  
4                NaN  
5                NaN  
6                NaN  
7                NaN  
8                NaN  
9         

In [33]:
class_1.shape

(550, 149)

In [35]:
class_2.shape

(558, 149)

In [36]:
print("I p-value is high or KS stat is low, we say that the two distributions are the same")
df_stat_ks.head()

I p-value is high or KS stat is low, we say that the two distributions are the same


,KS STAT,p-value
lh_S_orbital-H_Shaped_volume,0.260534,0.0
lh_G_and_S_subcentral_volume,0.256357,0.0
rh_G_precuneus_volume,0.260143,0.0
rh_G_temp_sup-Plan_tempo_volume,0.253164,0.0
rh_G_and_S_subcentral_volume,0.255523,0.0


In [38]:
df_stat_mw.head()

,MW STAT,p-value
lh_G_insular_short_volume,80639.5,7.507342e-43
rh_G_insular_short_volume,85495.0,1.373872e-37
rh_G_and_S_cingul-Ant_volume,86312.5,9.743414e-37
lh_S_temporal_sup_volume,88040.0,5.661740e-35
rh_S_temporal_sup_volume,88592.0,2.028197e-34


In [39]:
df_stat_validation_2.head()

,t(Xi),p-value
lh_G_insular_short_volume,14.904734,6.413012e-46
rh_G_insular_short_volume,13.713403,1.210018e-39
rh_G_and_S_cingul-Ant_volume,13.252767,2.581346e-37
rh_S_temporal_sup_volume,13.175475,6.267495e-37
lh_S_temporal_sup_volume,13.175040,6.298820e-37


In [40]:
df_stat.head()

,t(Xi),p-value
lh_G_and_S_frontomargin_volume,10.172561,0.0
rh_G_oc-temp_med-Lingual_volume,7.633197,0.0
rh_G_oc-temp_med-Parahip_volume,9.418678,0.0
rh_G_orbital_volume,12.086809,0.0
rh_G_pariet_inf-Angular_volume,11.626642,0.0
